In [ ]:
import panel as pn
pn.extension()

The ``FileDownload`` widget allows downloading a file on the frontend by sending the file data to the browser either on initialization (if ``embed=True``) or when the button is clicked.

Discover more on using widgets to add interactivity to your applications in the [how-to guides on interactivity](../how_to/interactivity/index.md). Alternatively, learn [how to set up callbacks and (JS-)links between parameters](../../how_to/links/index.md) or [how to use them as part of declarative UIs with Param](../../how_to/param/index.html).

#### Parameters:

For details on other options for customizing the component see the [layout](../../how_to/layout/index.md) and [styling](../../how_to/styling/index.md) how-to guides.

##### Core

* **``auto``** (boolean):  Whether to download the file the initial click (if `True`) or when clicking a second time (or via the right-click Save file menu).
* **``callback``** (callable): A callable that returns a file or file-like object (takes precedence over `file` if set). 
* **``embed``** (boolean):  Whether to embed the data on initialization.
* **``file``** (str or file-like object):  A path to a file or a file-like object.
* **``filename``** (str): The filename to save the file as.

##### Display

* **``button_type``** (str): A button theme; should be one of ``'default'`` (white), ``'primary'`` (blue), ``'success'`` (green), ``'info'`` (yellow), or ``'danger'`` (red)
* **``label``** (str): A custom label for the download button (by default uses the filename)
* **``name``** (str): The title of the widget

___

The `FileDownload` widget accepts a path to a file or a file-like object (with a `.read` method) if the latter is provided a `filename` must also be set. By default (`auto=True` and `embed=False`) the file is only transferred to the browser after the button is clicked (this requires a live-server or notebook kernel):

In [ ]:
file_download = pn.widgets.FileDownload(file='FileDownload.ipynb', filename='custom_filename.ipynb')

file_download

The file data may also be embedded immediately using `embed` parameter, this allows using the widget even in a static export:

In [ ]:
pn.widgets.FileDownload(file='FileDownload.ipynb', embed=True)

If `auto=False` is set the file will not be downloaded on the initial click but will change the label from "Transfer <file>" to "Download <file>" once the data has been synced. This offers an opportunity to download using the `Save as` dialog once the data has been transferred.

In [ ]:
pn.widgets.FileDownload(
    file='FileDownload.ipynb', button_type='success', auto=False,
    embed=False, name="Right-click to download using 'Save as' dialog"
)

The `FileDownload` widget may also be given a file-like object, e.g. here we save a pandas DataFrame as a CSV to a StringIO object and pass that to the widget:

In [ ]:
from bokeh.sampledata.autompg import autompg

from io import StringIO
sio = StringIO()
autompg.to_csv(sio)
sio.seek(0)

pn.widgets.FileDownload(sio, embed=True, filename='autompg.csv')

If you want to generate the file dynamically, e.g. because it depends on the parameters of some widget you can also supply a callback (which may be decorated with the widgets and/or parameters it depends on):

In [ ]:
years_options = list(autompg.yr.unique())
years = pn.widgets.MultiChoice(
    name='Years', options=years_options, value=[years_options[0]], margin=(0, 20, 0, 0)
)
mpg = pn.widgets.RangeSlider(
    name='Mile per Gallon', start=autompg.mpg.min(), end=autompg.mpg.max()
)

def filtered_mpg(yrs, mpg):
    df = autompg
    if years.value:
        df = autompg[autompg.yr.isin(yrs)]
    return df[(df.mpg >= mpg[0]) & (df.mpg <= mpg[1])]

def filtered_file(yr, mpg):
    df = filtered_mpg(yr, mpg)
    sio = StringIO()
    df.to_csv(sio)
    sio.seek(0)
    return sio

fd = pn.widgets.FileDownload(
    callback=pn.bind(filtered_file, years, mpg), filename='filtered_autompg.csv'
)

pn.Column(
    pn.Row(years, mpg),
    fd,
    pn.panel(pn.bind(filtered_mpg, years, mpg), width=600),
    width=600
)